In [2]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from keras.layers import Bidirectional
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import datetime
from numpy import hstack, vstack
from numpy import array
from sklearn.linear_model import SGDClassifier as SGD
from keras.models import load_model

In [4]:
model = load_model('Multiple_Parallel.h5')

Metal device set to: Apple M2


2022-11-12 18:56:59.378356: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-12 18:56:59.379820: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [17]:
#obtain list of dates
directory = "New York Knicks"
list_dates = []
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        list_dates.append(filename)

In [18]:
# create dictionary 
final_data = {}

In [19]:
# for minutes

minutes = {}

opponents = {}
# reads in each csv
for i in list_dates:
    data = pd.read_csv(f"{directory}/{i}", index_col=0)
    #gathers data and adds to dict
    date = data["Date"].tolist()[0]

    min_data = data.copy()



    minute = data["MIN"].tolist()

    minute_array = []

    for i in minute:
        time = i.split(":")

        minute_array.append(int(time[0]) + (int(time[0]) / 60))

    # (error for now) opponents[date] = data["Opponent"][0]

    data["MIN"] = minute_array

    data[date] = data["Score"]
    min_data[date] = data["MIN"]

    final_data[date] = data[date]
    minutes[date] = min_data[date]

# convert dict to dataframe
data_df = pd.DataFrame(final_data)

# convert dict to dataframe
min_df = pd.DataFrame(minutes)

opp = pd.DataFrame.transpose(pd.DataFrame.from_dict(opponents, orient="index"))
data_df = data_df.append(opp)

tester = pd.DataFrame.transpose(data_df)

tester["Datetime"] = pd.to_datetime(data_df.columns.tolist()).tolist()

tester = tester.sort_values(by="Datetime")

columns = tester.columns.tolist()[:-1]

tester_min = pd.DataFrame.transpose(min_df)

tester_min["Datetime"] = pd.to_datetime(min_df.columns.tolist()).tolist()

tester_min = tester_min.sort_values(by="Datetime")

columns = tester_min.columns.tolist()[:-1]

/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_38544/1099516675.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = data_df.append(opp)


In [20]:
add = {}

values = {}

diff = {}

columns = tester.columns.tolist()[:-1]

#iterate through index
for column in columns:
    
    if column not in add:
        add[column] = []
        
        values[column] = []
        
        diff[column] = []
    
    print(column)
    
    # split sequence
    def split_sequence(sequences, n_steps):
        X, y = list(), list()
        for i in range(len(sequences)):
            # find end of pattern 
            end_ix = i + n_steps
            #check if beyond sequence to break
            if end_ix > len(sequences) - 1:
                break
            #gather input and output parts of the pattern
            seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
            X.append(seq_x)
            y.append(seq_y)
        return array(X), array(y)
        # sequence
    
        # sequence
    df = tester[[column]]
    
    df["Min"] = tester_min[column].tolist()
    
    name = df.columns[0]
    
    rent_rate = array(df[name])
    
    # number of steps
    rent_rate = array(tester[column].dropna())
    predict = rent_rate
    eff = array(df["Min"].dropna())
    
    
    n_steps = 3
    if len(rent_rate) < 5:
        continue   
    
    
    # convert to [rows, columns] structure
    rent_rate = rent_rate.reshape((len(rent_rate), 1))
    eff = eff.reshape((len(eff), 1))
    predict = predict.reshape((len(predict), 1))
    
    # horizontally stack columns
    dataset = hstack((rent_rate, eff))    
        
    X, y = split_sequence(dataset, n_steps)
    
    X = X.astype(float)
    # summarize the data
    for i in range(len(X)):
        print(X[i], y[i])
        
    # reshape from [samples, timesteps] into [samples, timesteps, features]
    n_features = X.shape[2]
    
    rent = rent_rate[-3:]
    
    eff_pred = eff[-3:]
    
    # demonstrate prediction
    x_input = array([[rent[0], eff_pred[0]], [rent[1], eff_pred[1]], [rent[2], eff_pred[2]]])
    x_input = x_input.reshape((1, n_steps, n_features))
    yhat = model.predict(x_input, verbose=0)
    add[column].append(yhat.tolist()[0][0])

Cam Reddish
[[36.75       28.46666667]
 [10.         18.3       ]
 [12.5        14.23333333]] [10.25 24.4 ]
[[10.         18.3       ]
 [12.5        14.23333333]
 [10.25       24.4       ]] [ 6.         11.18333333]
[[12.5        14.23333333]
 [10.25       24.4       ]
 [ 6.         11.18333333]] [11.         16.26666667]
[[10.25       24.4       ]
 [ 6.         11.18333333]
 [11.         16.26666667]] [ 8.         14.23333333]
[[ 6.         11.18333333]
 [11.         16.26666667]
 [ 8.         14.23333333]] [20.   15.25]
[[11.         16.26666667]
 [ 8.         14.23333333]
 [20.         15.25      ]] [13.5        37.61666667]
[[ 8.         14.23333333]
 [20.         15.25      ]
 [13.5        37.61666667]] [ 9.         28.46666667]
[[20.         15.25      ]
 [13.5        37.61666667]
 [ 9.         28.46666667]] [ 1.5        17.28333333]
Derrick Rose
[[13.5        12.2       ]
 [26.25       16.26666667]
 [ 3.5         8.13333333]] [17.5 12.2]
[[26.25       16.26666667]
 [ 3.5        

/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_38544/702853107.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Min"] = tester_min[column].tolist()
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_38544/702853107.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Min"] = tester_min[column].tolist()
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_38544/702853107.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

Evan Fournier
[[29.         32.53333333]
 [16.75       19.31666667]
 [14.5        26.43333333]] [22.75 24.4 ]
[[16.75       19.31666667]
 [14.5        26.43333333]
 [22.75       24.4       ]] [16.5        20.33333333]
[[14.5        26.43333333]
 [22.75       24.4       ]
 [16.5        20.33333333]] [24.75       28.46666667]
[[22.75       24.4       ]
 [16.5        20.33333333]
 [24.75       28.46666667]] [ 6.5 18.3]
[[16.5        20.33333333]
 [24.75       28.46666667]
 [ 6.5        18.3       ]] [ 9.         14.23333333]
[[24.75       28.46666667]
 [ 6.5        18.3       ]
 [ 9.         14.23333333]] [ 7.25       10.16666667]
[[ 6.5        18.3       ]
 [ 9.         14.23333333]
 [ 7.25       10.16666667]] [10.75 12.2 ]
[[ 9.         14.23333333]
 [ 7.25       10.16666667]
 [10.75       12.2       ]] [14.         20.33333333]
Immanuel Quickley
[[ 3.75       16.26666667]
 [39.25       27.45      ]
 [20.5        23.38333333]] [18.75 18.3 ]
[[39.25       27.45      ]
 [20.5        23.38

/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_38544/702853107.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Min"] = tester_min[column].tolist()
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_38544/702853107.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Min"] = tester_min[column].tolist()
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_38544/702853107.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

Miles McBride
Mitchell Robinson
[[10.5        12.2       ]
 [18.75       23.38333333]
 [29.5        29.48333333]] [32.5        37.61666667]
[[18.75       23.38333333]
 [29.5        29.48333333]
 [32.5        37.61666667]] [21.75       20.33333333]
[[29.5        29.48333333]
 [32.5        37.61666667]
 [21.75       20.33333333]] [16.75       20.33333333]
[[32.5        37.61666667]
 [21.75       20.33333333]
 [16.75       20.33333333]] [33.5 18.3]
[[21.75       20.33333333]
 [16.75       20.33333333]
 [33.5        18.3       ]] [ 4.         11.18333333]
Obi Toppin
[[ 6.75       13.21666667]
 [28.5        20.33333333]
 [18.         13.21666667]] [17.25       13.21666667]
[[28.5        20.33333333]
 [18.         13.21666667]
 [17.25       13.21666667]] [19.25       16.26666667]
[[18.         13.21666667]
 [17.25       13.21666667]
 [19.25       16.26666667]] [13.   15.25]
[[17.25       13.21666667]
 [19.25       16.26666667]
 [13.         15.25      ]] [16.75 18.3 ]
[[19.25       16.266666

/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_38544/702853107.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Min"] = tester_min[column].tolist()
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_38544/702853107.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Min"] = tester_min[column].tolist()
/var/folders/g3/164q6cz52jv0sjllnsg7jdsm0000gn/T/ipykernel_38544/702853107.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

In [21]:
# convert dict to dataframe
Data = pd.DataFrame.from_dict(add, orient='index')
Data

,0
Cam Reddish,14.966345
Derrick Rose,13.389699
Evan Fournier,16.813444
Immanuel Quickley,21.482470
Isaiah Hartenstein,8.990362
Jalen Brunson,15.057981
Jericho Sims,24.874460
Julius Randle,41.487286
Miles McBride,NaN
Mitchell Robinson,18.154642


In [22]:
# convert dataframe to csv
mypath = "Final_Score/LSTM"
if not os.path.isdir(mypath):
    os.makedirs(mypath)
Data.to_csv(f"{mypath}/{directory}.csv", index=True)